# Knowledge Distillation

**Author:** [Marcus Rüb](https://www.linkedin.com/in/marcus-r%C3%BCb-3b07071b2/)<br>
**Date created:** 2021/03/22<br>
**Description:** Implementation of classical Knowledge Distillation.

## Einführung in Knowledge Distillation

Knowledge Distillation ist ein Verfahren zur Modellkomprimierung, bei dem ein kleines (Schüler-)Modell so trainiert wird, dass es zu einem großen vortrainierten (Lehrer-)Modell passt. Das Wissen wird vom Lehrermodell auf das Schülermodell übertragen, indem eine Verlustfunktion minimiert wird, die darauf abzielt, die aufgeweichten Logits des Lehrermodells sowie die Ground-Truth-Etiketten abzugleichen. Die Logits werden durch Anwendung einer "Temperatur"-Skalierungsfunktion im Softmax aufgeweicht, wodurch die Wahrscheinlichkeitsverteilung effektiv geglättet wird und die vom Lehrer gelernten Beziehungen zwischen den Klassen sichtbar werden.



**Reference:**

- [Hinton et al. (2015)](https://arxiv.org/abs/1503.02531)

## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


## Construct `Distiller()` class

Die benutzerdefinierte Klasse `Distiller()` überschreibt die `Model`-Methoden `train_step`, `test_step` und `compile()`. Um den Distiller verwenden zu können, benötigen wir:

- Ein trainiertes Lehrermodell
- Ein Schüler-Modell zum Trainieren
- Eine Schüler-Verlustfunktion für die Differenz zwischen Schüler-Vorhersagen und Groundtruth
- Eine Destillationsverlustfunktion, zusammen mit einer `Temperatur`, auf die Differenz zwischen den weichen Studentenvorhersagen und den weichen Lehreretiketten
- Ein "Alpha"-Faktor zur Gewichtung der Studenten- und Destillationsverluste
- Ein Optimierer für den Schüler und (optionale) Metriken zur Leistungsbewertung

Bei der Methode "train_step" führen wir einen Vorwärtsdurchlauf von Lehrer und Schüler durch, berechnen den Verlust mit der Gewichtung von "student_loss" und "distillation_loss" mit "alpha" bzw. "1 - alpha" und führen den Rückwärtsdurchlauf durch. Hinweis: Es werden nur die Schülergewichte aktualisiert, und daher werden nur die Gradienten für die Schülergewichte berechnet.
In der Methode `test_step` wird das Studentenmodell auf dem bereitgestellten Datensatz evaluiert.


In [ ]:

class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


## Create student and teacher models

Zunächst erstellen wir ein Lehrer-Modell und ein kleineres Schüler-Modell. Beide Modelle sind Faltungsneuronale Netze und werden mit `Sequential()` erstellt, können aber jedes Keras-Modell sein.

In [ ]:
# Create the teacher
teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="teacher",
)

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

## Vorbereiten des Datensatzes

Der Datensatz, der für das Training des Lehrers und die Destillation des Lehrers verwendet wird, ist [MNIST](https://keras.io/api/datasets/mnist/). Das Verfahren wäre für jeden anderen Datensatz, z. B. [CIFAR-10](https://keras.io/api/datasets/cifar10/), mit einer geeigneten Wahl der Modelle äquivalent. Sowohl der Schüler als auch der Lehrer werden auf dem Trainingsset trainiert und auf dem Testset evaluiert.

In [ ]:
# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))



## Den Lehrer ausbilden

Bei der Wissensdestillation gehen wir davon aus, dass der Lehrer trainiert und festgelegt ist. Wir beginnen also damit, das Lehrermodell auf dem Trainingsset auf die übliche Weise zu trainieren.

In [ ]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=5)
teacher.evaluate(x_test, y_test)

## Lehrer zu Schüler destillieren

Wir haben das Lehrermodell bereits trainiert und müssen nur noch eine `Distiller(student, teacher)`-Instanz initialisieren, sie mit den gewünschten Verlusten, Hyperparametern und Optimierer `kompilieren()` und den Lehrer auf den Schüler destillieren.

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=3)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

## Schüler von Grund auf trainieren zum Vergleich

Wir können auch ein äquivalentes Studentenmodell von Grund auf ohne den Lehrer trainieren, um den durch die Wissensdestillation erzielten Leistungsgewinn zu bewerten.

In [ ]:
# Train student as doen usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=3)
student_scratch.evaluate(x_test, y_test)

Wenn der Lehrer für 5 volle Epochen trainiert wird und der Student für 3 volle Epochen auf diesem Lehrer destilliert wird, sollten Sie in diesem Beispiel einen Leistungsschub im Vergleich zum Training des gleichen Studentenmodells von Grund auf und sogar im Vergleich zum Lehrer selbst erleben. Sie sollten erwarten, dass der Lehrer eine Genauigkeit von etwa 97,6 % hat, der von Grund auf trainierte Student sollte etwa 97,6 % haben und der destillierte Student sollte etwa 98,1 % haben. Entfernen oder probieren Sie verschiedene Seeds aus, um unterschiedliche Gewichtungsinitialisierungen zu verwenden.